<a href="https://colab.research.google.com/github/frizuma3/21013159_DataAnalytics/blob/main/DNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# needed to create the data frame
import pandas as pd

# needed to help with speedy maths based calculations
import numpy as np

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/frizuma3/21013159_DataAnalytics/refs/heads/main/assignment2dnndata.csv', index_col=0)

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.17.1


In [3]:
# make sure we have our data by printing it out
df[:5]
# df #all

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,...,Mon,Sat,Sun,Thu,Tue,Wed,year,temp,dewp,NUM_COLLISIONS
1,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,2012,83.6,63.0,0.520270
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,2012,80.3,54.1,0.578829
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,2012,79.8,56.7,0.804054
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,2012,81.8,65.6,0.281532
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,2012,86.7,64.3,0.639640


In [4]:
dnn_input_data = [df["year"], df["temp"], df["dewp"], df["Sat"], df["Sun"], df["Mon"], df["Tue"], df["Wed"], df["Thu"], df["Fri"], df["Jan"], df["Feb"], df["Mar"], df["Apr"], df["May"], df["Jun"], df["Jul"], df["Aug"], df["Sep"], df["Oct"], df["Nov"], df["Dec"], df["NUM_COLLISIONS"]]
headers = ["year","temp", "dewp", "Sat","Sun","Mon","Tue","Wed","Thu","Fri","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec","NUM_COLLISIONS"]
df_dnn_input = pd.concat(dnn_input_data, axis=1, keys=headers)
df_dnn_input.head()

,year,temp,dewp,Sat,Sun,Mon,Tue,Wed,Thu,Fri,...,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,NUM_COLLISIONS
1,2012,83.6,63.0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.520270
2,2012,80.3,54.1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0.578829
3,2012,79.8,56.7,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0.804054
4,2012,81.8,65.6,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0.281532
5,2012,86.7,64.3,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0.639640


In [5]:
training_dataset = df_dnn_input.sample(frac=0.8, random_state=0)
test_dataset = df_dnn_input.drop(training_dataset.index)

In [6]:
training_features = training_dataset.copy()
test_features = test_dataset.copy()

training_labels = training_features.pop('NUM_COLLISIONS')
test_labels = test_features.pop('NUM_COLLISIONS')

In [7]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_COLLISIONS = 1.0

In [8]:
training_labels = training_labels/SCALE_NUM_COLLISIONS
test_labels = test_labels/SCALE_NUM_COLLISIONS

In [9]:
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features))

In [10]:
# This is the only difference, instead of a single layer, we have our normalisation layer (22 inputs), 2 layers of 48, with 1 output. The 48 can be adjusted to improve the net.
dnn_model_1 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_1.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

In [11]:
%%time
history = dnn_model_1.fit(
    training_features,
    training_labels,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 19.3 s, sys: 1.05 s, total: 20.4 s
Wall time: 26.8 s


In [12]:
# remember, we want to minimise this. The model with the lowest is the best.
dnn_model_1_results = dnn_model_1.evaluate(test_features, test_labels, verbose=0)
print(dnn_model_1_results)

0.122939333319664


In [13]:
# make sure the labels match up with the dataframe from earlier.
input_1 = pd.DataFrame.from_dict(data =
				{
         'year' : [2020, 2020, 2020],
         'temp' : [83.6, 80.3, 79.8],
         'dewp' : [63, 54.1, 56.7],
         'Sat' : [0,0,0],
         'Sun' : [1,0,0],
         'Mon' : [0,0,0],
         'Tue' : [0,1,0],
         'Wed' : [0,0,1],
         'Thu' : [0,0,0],
         'Fri' : [0,0,0],
         'Jan' : [0,0,0],
         'Feb' : [0,0,0],
         'Mar' : [0,0,0],
         'Apr' : [0,0,0],
         'May' : [0,0,0],
         'Jun' : [0,0,0],
         'Jul' : [1,1,1],
         'Aug' : [0,0,0],
         'Sep' : [0,0,0],
         'Oct' : [0,0,0],
         'Nov' : [0,0,0],
         'Dec' : [0,0,0],
        })


In [14]:
linear_day_predictions = dnn_model_1.predict(input_1[:3])*SCALE_NUM_COLLISIONS
linear_day_predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step


array([[0.24169847],
       [0.6549182 ],
       [0.57167774]], dtype=float32)

# Analysis of Predictions

# Impact of Features:
The prediction values indicate a general trend where the temperature and day of the week appear to influence the predictions.
The highest prediction (618 collisions) is associated with a Wednesday in July with moderate temperature (79.8degF) and dew point (56.7degF).
Conversely, the lowest prediction (291 collisions) is linked to a Sunday in July with a high temperature (83.6degF) and higher dew point (63degF). The relatively lower prediction on a Sunday could reflect lower commuter activity compared to weekdays.

# Non-linear Relationships:
The differences in predictions across the three examples show the ability of the DNN to model non-linear relationships between the input features and the target variable. For instance, temperature and dew point seem to interact differently depending on the day, reflecting the fact that conditions like these may have varied effects depending on other factors like human behaviour (weekdays vs. weekends).

# Feature Importance:
The model's predictions suggest that the day of the week might play a significant role, as each example corresponds to different weekdays. The Sunday prediction is notably lower, aligning with what we know from earlier correlations, where weekdays showed increased collision activity compared to weekends.

# Seasonal Influence:
The variance in predicted collisions, despite the similar seasonal context, suggests that temperature differences, in combination with other day-specific factors, influence the outcome. It implies that even in the same month(July), temperature and day-related patterns still significantly affect collision frequency.


  The DNN model, with two hidden layers of 48 units each, has been trained to predict the number of collisions based on a variety of features, including temperature, dew point, day of the week and month. The evaluation of the model on test data yielded a mean absolute error (MAE) of 0.1188 (i.e., approximately 119 collisions when scaled).

The predictions for the input examples demonstrate the influence of day of the week and temperature on collision counts. The lowest prediction is for a Sunday, potentially reflecting reduced traffic activity. The higher predictions are for Tuesday and Wednesday, suggesting that these midweek days might have more active traffic patterns leading to more collisions. The model effectively captures these nuances, showing the benefit of using a DNN to model complex relationships among the multiple input features.

For the first, you can see below, we are in August, Saturday (we have a monday), 2009. We have a lower temperature and lower windspeed. The difference in day is likely to account for the higher actual number of trips.

205	0	1	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	2009	64.9	4.2	0.700748221591537

For second (December):

337	0	0	1	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	2009	35.2	6.6	0.62902717790116

For third (February):

28	0	0	0	1	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	2009	38.1	3.6	0.675798566550843

The results overall look reasonable. You can look into this further.


Considerations for Assignment. I would likely use another method to either standardise or normalise num_trips. And perhaps not bother and scale them in here with SCALE_NUM_TRIPS with a generally large number.

Obviously, you will likely want to have a number of variations but there is no reason you can't use most of the data given. Remember, the DNN is trying to solve a complex relationship, not a linear one.

Other things to consider when doing this would be to take validation. Of course, in the assignment you will have a much larger range of data i.e. from some date x to y.

This will give you more data. Taking real validation data that hasn't been shown to the model will give real results for you to check against rather than what I have done here (which is just for information).